In [1]:
%load_ext autoreload
%autoreload 2
import os
if not os.path.exists('./modules') and not os.path.exists('modules.zip'):
    from google.colab import files
    uploaded = files.upload()
if not os.path.exists('./modules') and os.path.exists('modules.zip'):
    os.system('unzip modules.zip -d .')

if not os.path.exists('./Models') and not os.path.exists('Models.zip'):
    from google.colab import files
    uploaded = files.upload()
if not os.path.exists('./Models') and os.path.exists('Models.zip'):
    os.system('unzip Models.zip -d .')

!pip3 install optuna
!pip3 install mne
import torch
import torch.nn as nn
import optuna
from modules import Trainer
from modules.competition_dataset_one_per_trial import EEGDataset
from modules.utils import evaluate_model
import random
import numpy as np
from torch.utils.data import DataLoader
from Models.lfann_dann_gpt_generated import LFANN_DANN
import torch.nn.functional as F
from torch.autograd import Function

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:
# from google.colab import drive

# drive.mount("/content/drive/")
# try:
#     print()
#     os.symlink("/content/drive/MyDrive/ai_data/eeg_detection/data", "./data")
#     os.symlink("/content/drive/MyDrive/ai_data/eeg_detection/checkpoints", "./checkpoints")
# except:
#     pass

data_path = "./data/mtcaic3"
model_path = "./checkpoints/mi/models/the_honored_one.pth"
optuna_db_path = "./checkpoints/mi/optuna/the_honored_one.db"

In [3]:
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [4]:
batch_size = 32
eeg_channels = ["C3", "PZ", "OZ"] # ["C3", "CZ", "C4"]

dataset_train = EEGDataset(
    data_path=data_path,
    task="MI",
    split="train",
    tmin=0.5,  # skip first 0.5 s (125 samples)
    win_len=int(4.5 * 250),  # 4.5 s window → 1125 samples
    read_labels=True,
    eeg_channels=eeg_channels,
)

dataset_val = EEGDataset(
    data_path=data_path,
    task="MI",
    split="validation",
    tmin=0.5,
    win_len=int(4.5 * 250),
    read_labels=True,
    eeg_channels=eeg_channels,
)

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(dataset_val,   batch_size=batch_size)

data shape: torch.Size([2400, 3, 1125]), classes shape: torch.Size([2400, 2])
data shape: torch.Size([50, 3, 1125]), classes shape: torch.Size([50, 2])
data shape: torch.Size([50, 3, 1125]), classes shape: torch.Size([50, 2])


In [6]:
dataset_train[0][0].shape

torch.Size([3, 1125])

In [8]:
# ---------------- Gradient Reversal Layer ---------------- #
class GradientReversalFunction(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None

class GradientReversal(nn.Module):
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha

    def forward(self, x):
        return GradientReversalFunction.apply(x, self.alpha)

# ---------------- LSTM Head ---------------- #
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        # x: B x seq_len x feat_dim
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# ---------------- EEG Feature Extractor ---------------- #
class EEGFeatureExtractor(nn.Module):
    def __init__(self, n_electrodes, kernLength, F1, D, F2, dropout):
        super().__init__()
        # For input B x C x T, apply 2D convs on [C x T]
        self.block = nn.Sequential(
            # Temporal conv across time
            nn.Conv2d(1, F1, (1, kernLength), padding=(0, kernLength//2), bias=False),
            nn.BatchNorm2d(F1),
            # Depthwise spatial conv across electrodes
            nn.Conv2d(F1, F1 * D, (n_electrodes, 1), groups=F1, bias=False),
            nn.BatchNorm2d(F1 * D),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout),
            # Separable conv
            nn.Conv2d(F1 * D, F1 * D, (1, 16), padding=(0, 8), groups=F1 * D, bias=False),
            nn.Conv2d(F1 * D, F2, 1, bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # x: B x C x T
        x = x.unsqueeze(1)  # B x 1 x C x T
        x = self.block(x)   # B x F2 x 1 x T_sub
        x = x.squeeze(2)    # B x F2 x T_sub
        x = x.permute(0, 2, 1)  # B x T_sub x F2
        return x

# ---------------- DANN SSVEP Classifier ---------------- #
class DANN_SSVEPClassifier(nn.Module):
    def __init__(self,
                 n_electrodes=16,
                 out_dim=4,
                 dropout=0.25,
                 kernLength=256,
                 F1=96,
                 D=1,
                 F2=96,
                 hidden_dim=100,
                 layer_dim=1,
                 grl_alpha=1.0,
                 domain_classes=2):
        super().__init__()
        self.feature_extractor = EEGFeatureExtractor(
            n_electrodes, kernLength, F1, D, F2, dropout
        )
        feat_dim = F2
        # label predictor (LSTM over time)
        self.label_lstm = LSTMModel(feat_dim, hidden_dim, layer_dim, out_dim)
        # domain predictor
        self.grl = GradientReversal(alpha=grl_alpha)
        self.domain_fc = nn.Sequential(
            nn.Linear(feat_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, domain_classes)
        )

    def forward(self, x):
        # x: B x C x T
        feat_seq = self.feature_extractor(x)        # B x T_sub x feat_dim
        class_out = self.label_lstm(feat_seq)
        # domain prediction from time-averaged features
        feat_avg = feat_seq.mean(dim=1)             # B x feat_dim
        rev_feat = self.grl(feat_avg)
        domain_out = self.domain_fc(rev_feat)
        return class_out, domain_out

# Example usage
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dummy_x = torch.randn(5, 3, 256).to(device)  # B x C=3 x T=256
    model = DANN_SSVEPClassifier(
        n_electrodes=3,
        out_dim=2,
        kernLength=256,
        F1=32,
        D=3,
        F2=32,
        hidden_dim=256,
        layer_dim=3,
        grl_alpha=0.1,
        domain_classes=2
    ).to(device)
    class_logits, domain_logits = model(dummy_x)
    print(class_logits.shape, domain_logits.shape)  # (5,2), (5,2)


print("shit", dataset_train[0][0].shape)
C, T = dataset_train[0][0].shape
model = DANN_SSVEPClassifier(
    n_electrodes=C,
    dropout=0.26211635308091535,
    out_dim=2,
    kernLength=8,
    F1 = 32,
    D = 3,
    F2 = 32,
    hidden_dim=256,
    layer_dim=2,
    domain_classes=30,
).to(device)

inp = torch.randn(3, C, T).to(device)
out = model(inp)
out

torch.Size([5, 2]) torch.Size([5, 2])
shit torch.Size([3, 1125])


(tensor([[0.0408, 0.0398],
         [0.0477, 0.0364],
         [0.0410, 0.0362]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[-0.2126, -0.2425,  0.1154,  0.1786, -0.2548, -0.1960,  0.1415, -0.0603,
           0.1728, -0.0764,  0.0259,  0.1047, -0.0516, -0.5328, -0.0760, -0.4604,
           0.0933, -0.2219, -0.3482,  0.0374, -0.0108,  0.2840,  0.1244,  0.0371,
          -0.1087,  0.0408, -0.1810,  0.5047,  0.0199,  0.3573],
         [-0.1168, -0.3141,  0.0854,  0.0682,  0.1217,  0.1356,  0.0760, -0.1201,
           0.0260, -0.0507,  0.0843,  0.1459, -0.4855, -0.4441,  0.2663,  0.2306,
           0.2954, -0.3287, -0.0633,  0.1073, -0.0595, -0.2343,  0.0434, -0.1195,
           0.0137, -0.1080, -0.1419,  0.2733,  0.0249,  0.2188],
         [-0.3697, -0.3438,  0.1704, -0.1303,  0.0367, -0.0027,  0.1749,  0.0914,
          -0.1998, -0.0855,  0.1553,  0.1267, -0.1130, -0.5895,  0.0901,  0.0517,
          -0.0385, -0.2807, -0.4176, -0.0926,  0.0929, -0.1906,  0.1936, -0.0084,
      

In [9]:
avg_losses_label = []
avg_losses_domain = []
val_label_accuracies = []
val_domain_accuracies = []
train_label_accuracies = []
train_domain_accuracies = []

In [10]:
try:
    model.load_state_dict(torch.load(model_path, weights_only=True))
except Exception:
    print("skipping model loading...")


opt = torch.optim.Adam(model.parameters(), lr=0.0003746351873334935)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt,
    mode="min",  # “min” if you want to reduce lr when the quantity monitored has stopped decreasing
    factor=0.5,  # new_lr = lr * factor
    patience=20,  # number of epochs with no improvement after which lr will be reduced
    threshold=1e-4,  # threshold for measuring the new optimum, to only focus on significant changes
    threshold_mode="rel",  # `'rel'` means compare change relative to best value. Could use `'abs'`.
    cooldown=0,  # epochs to wait before resuming normal operation after lr has been reduced
    min_lr=1e-6,  # lower bound on the lr
)

epochs = 4000

for epoch in range(epochs):
    avg_loss_label = 0
    avg_loss_domain = 0
    correct_label = 0
    correct_domain = 0
    total = 0
    model.train()

    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device).to(torch.int64) # shape: [Bx2], 0: label, 1: domain
        y_labels = y[:, 0]
        y_subj = y[:, 1]

        y_pred_labels, y_pred_domain = model(x)

        loss_label = criterion(y_pred_labels, y_labels)
        loss_domain = criterion(y_pred_domain, y_subj)
        loss = loss_label + loss_domain
        
        opt.zero_grad()
        loss.backward()
        opt.step()

        avg_loss_label += loss_label.item()
        avg_loss_domain += loss_domain.item()

        # Accuracy calculation
        _, pred_labels = torch.max(y_pred_labels, 1)
        _, pred_domains = torch.max(y_pred_domain, 1)
        correct_label += (pred_labels == y_labels).sum().item()
        correct_domain += (pred_domains == y_subj).sum().item()
        total += y_labels.size(0)

    avg_loss_label /= len(train_loader)
    avg_loss_domain /= len(train_loader)
    avg_losses_label.append(avg_loss_label)
    avg_losses_domain.append(avg_loss_domain)
    train_label_acc = 100.0 * correct_label / total
    train_domain_acc = 100.0 * correct_domain / total
    train_label_accuracies.append(train_label_acc)
    train_domain_accuracies.append(train_domain_acc)
    # scheduler.step(avg_loss_label)

    if epoch % 5 == 0:
        label_evaluation, domain_evaluation = evaluate_model(model, val_loader, device)
        val_label_accuracies.append(label_evaluation)
        val_domain_accuracies.append(domain_evaluation)
        model.cpu()
        torch.save(model.state_dict(), model_path)
        model.to(device)
        print(f"{epoch}: avg_loss_label: {avg_loss_label:.3F}, avg_loss_domain: {avg_loss_domain:.3F}, train_label_acc: {train_label_acc:.2f}%, train_domain_acc: {train_domain_acc:.2f}%, label_evaluation: {(label_evaluation*100):.2f}, domain_evaluation: {(domain_evaluation*100):.2f}, lr: {opt.param_groups[0]['lr']}")

0: avg_loss_label: 0.693, avg_loss_domain: 3.361, train_label_acc: 49.75%, train_domain_acc: 3.62%, label_evaluation: 44.00, domain_evaluation: 0.00, lr: 0.0003746351873334935
5: avg_loss_label: 0.694, avg_loss_domain: 3.264, train_label_acc: 49.71%, train_domain_acc: 6.88%, label_evaluation: 44.00, domain_evaluation: 0.00, lr: 0.0003746351873334935
5: avg_loss_label: 0.694, avg_loss_domain: 3.264, train_label_acc: 49.71%, train_domain_acc: 6.88%, label_evaluation: 44.00, domain_evaluation: 0.00, lr: 0.0003746351873334935
10: avg_loss_label: 0.693, avg_loss_domain: 3.440, train_label_acc: 49.58%, train_domain_acc: 3.96%, label_evaluation: 52.00, domain_evaluation: 0.00, lr: 0.0003746351873334935
10: avg_loss_label: 0.693, avg_loss_domain: 3.440, train_label_acc: 49.58%, train_domain_acc: 3.96%, label_evaluation: 52.00, domain_evaluation: 0.00, lr: 0.0003746351873334935
15: avg_loss_label: 0.693, avg_loss_domain: 3.620, train_label_acc: 50.04%, train_domain_acc: 4.62%, label_evaluation:

KeyboardInterrupt: 

In [ ]:
class CustomTrainer(Trainer):
    # This method is called by _objective during an Optuna trial
    def prepare_trial_run(self):
        assert isinstance(self.trial, optuna.Trial), "Trial not set!"

        # 1. Define Hyperparameters for this trial
        #    a. Data/Loader parameters
        window_length = self.trial.suggest_categorical("window_length", [128, 256, 640]) # e.g. 64*2, 64*4, 64*10
        batch_size = self.trial.suggest_categorical("batch_size", [32, 64])

        #    b. Model architecture parameters
        kernLength = self.trial.suggest_categorical("kernLength", [64, 128, 256])
        F1 = self.trial.suggest_categorical("F1", [8, 16, 32])
        D = self.trial.suggest_categorical("D", [1, 2, 3])
        F2 = self.trial.suggest_categorical("F2", [16, 32, 64]) # F2 must be F1 * D
        hidden_dim = self.trial.suggest_categorical("hidden_dim", [64, 128, 256])
        layer_dim = self.trial.suggest_categorical("layer_dim", [1, 2, 3])
        dropout = self.trial.suggest_float("dropout", 0.1, 0.6)
        
        #    c. Optimizer parameters
        lr = self.trial.suggest_float("lr", 1e-4, 1e-2, log=True)

        # 2. Prepare the data using these parameters
        super()._prepare_data(is_trial=True, batch_size=batch_size, window_length=window_length)
        
        assert self.dataset is not None, "Dataset was not created correctly"
        n_electrodes = self.dataset.datasets[0].data[0].shape[0] # Get shape from underlying dataset

        # 3. Build the model and optimizer
        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes, # Use value from data
            dropout=dropout,
            kernLength=kernLength,
            F1=F1,
            D=D,
            F2=F1 * D, # F2 is dependent on F1 and D
            hidden_dim=hidden_dim,
            layer_dim=layer_dim,
        ).to(self.device)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

    # This method is called by train() for the final run
    def prepare_final_run(self):
        # 1. Get the best hyperparameters from the completed study
        study = self._get_study()
        best_params = study.best_params
        
        # 2. Prepare data using the best params
        super()._prepare_data(is_trial=False) # is_trial=False handles getting params from study
        
        assert self.dataset is not None, "Dataset was not created correctly"
        n_electrodes = self.dataset.datasets[0].data[0].shape[0]

        # 3. Build the final model and optimizer
        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes,
            out_dim=2,
            dropout=best_params["dropout"],
            kernLength=best_params["kernLength"],
            F1=best_params["F1"],
            D=best_params["D"],
            F2=best_params["F1"] * best_params["D"],
            hidden_dim=best_params["hidden_dim"],
            layer_dim=best_params["layer_dim"],
        ).to(self.device)
        
        # Optional: Load pre-existing weights if you are resuming
        try:
            self.model.load_state_dict(torch.load(self.model_path))
            print(f"Loaded existing model weights from {self.model_path}")
        except Exception:
            print(f"No existing model weights found at {self.model_path}. Training from scratch.")
        
        lr = 0.00018182233882257615 # best_params["lr"]
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='max',        # “min” if you want to reduce lr when the quantity monitored has stopped decreasing
            factor=0.5,        # new_lr = lr * factor
            patience=20,        # number of epochs with no improvement after which lr will be reduced
            threshold=1e-4,    # threshold for measuring the new optimum, to only focus on significant changes
            threshold_mode='rel', # `'rel'` means compare change relative to best value. Could use `'abs'`.
            cooldown=0,        # epochs to wait before resuming normal operation after lr has been reduced
            min_lr=1e-6,       # lower bound on the lr
        )

trainer = CustomTrainer(
        data_path=data_path,
        optuna_db_path=optuna_db_path,
        model_path=model_path,
        train_epochs=500, # Final training epochs
        tune_epochs=50,   # Epochs per trial
        optuna_n_trials=50,
        task="mi",
        eeg_channels=eeg_channels,
        data_fraction=0.4
    )

In [ ]:
delete_existing = False
trainer.optimize(delete_existing)

In [ ]:
trainer.train()

In [ ]:
trainer._prepare_training(False)
trainer.model.eval()
f"test accuracy: {evaluate_model(trainer.model, trainer.eval_loader, device)}"